# 1141ML Final - DSM 2D

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np

device = torch.device("cpu")
## ---參數跟之前差不多

sigma_dsm=1.0
batch_size=2048
